# CAMBIO
Statistical analysis of CAMBI study results.
This workbench is intended to give a first overview of CAMBIO results.
We start from the (preliminary) output data file from CAMBIO. We first iterate over all numerical columns and perform a t test (to compare AMD and Control). We discard data from rows without valid numerical results or that are empty. We do not model left/right eye data, thus, this is just for a very first look and is not intended to be a final analyis. We then perform an analogous analysis on the categorical data using a chi-squared test.

### import necessary Python libraries (our library s called cambio)

In [1]:
from cambio import Cambio
from pathlib import Path

### Input path to data using dialog

In [2]:
# input() ## enter path to CAMBio_Tabelle

### Convert relative path to absolute path and initialize Cambio object

In [3]:
input_file = "../../../Documents/manuscripts/AMD/2025_07_24_CAMBio_Tabelle_für_Hannes_Urbat.txt"
input_path = Path(input_file).resolve()
cambio = Cambio(cambio_path=input_path)

### Perform t tests on all numerical columns

In [4]:
t_test_d_list = cambio.perform_t_tests()

### Print sorted results

In [5]:
for res in t_test_d_list:
    col = res['column']
    t = res['t_stat']
    p = res['p_val']
    print(f"{col:<20} t={t:>8.4f}  p={p:>10.4g}")
    for group, mean in res['diag']['means'].items():
        std = res['diag']['stds'][group]
        n = res['diag']['n_obs'][group]
        print(f"    {group:<10} mean={mean:.3f}  std={std:.3f}  n={n}")
    print()

Age at Admission     t=  7.8564  p= 7.616e-13
    AMD        mean=79.026  std=5.456  n=78
    Control    mean=70.631  std=7.991  n=84

FI Particle Counts CD9 (fraction of total) t=  2.9345  p=  0.005288
    AMD        mean=0.095  std=0.051  n=27
    Control    mean=0.061  std=0.031  n=23

Height (cm)          t= -2.5722  p=   0.01183
    AMD        mean=162.862  std=27.862  n=78
    Control    mean=171.202  std=6.865  n=84

FI Particle Counts CD81 (fraction of total) t=  2.4085  p=   0.02163
    AMD        mean=0.202  std=0.177  n=28
    Control    mean=0.117  std=0.061  n=26

Fluorescence Intensity C3 CD9 (ExoView) t=  2.2031  p=   0.03164
    AMD        mean=1.059  std=0.042  n=33
    Control    mean=1.039  std=0.029  n=29

Fluorescence per particle FH CD9 (ExoView) t=  2.1309  p=   0.03811
    AMD        mean=187.847  std=145.285  n=34
    Control    mean=127.879  std=70.454  n=29

FI Particle Counts CD41a (fraction of total) t=  2.0160  p=   0.05074
    AMD        mean=0.295  std=0